In [1]:
!pip install datasets
!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install transformers
!pip install torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-vcf5n08d
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whis

In [2]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mozillaorg/common-voice")

print("Path to dataset files:", path)

100%|██████████| 12.0G/12.0G [02:38<00:00, 81.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mozillaorg/common-voice/versions/2


In [4]:
import shutil

source_path = path  # Ścieżka do pobranych plików
destination = "/content"

shutil.move(source_path, destination)

print(f"Pliki zostały przeniesione do {destination}")


Pliki zostały przeniesione do /content


In [8]:
import os

sample_path = "/content/2/cv-other-test/cv-other-test/sample-000000.mp3"
if os.path.isfile(sample_path):
    print(f"Plik {sample_path} istnieje.")
    # Możesz również odtworzyć plik, aby upewnić się, że jest poprawny
    import IPython.display as ipd
    ipd.Audio(sample_path)
else:
    print(f"Plik {sample_path} nie istnieje.")


Plik /content/2/cv-other-test/cv-other-test/sample-000000.mp3 istnieje.


In [10]:
df = pd.read_csv('/content/2/cv-other-test.csv')

In [15]:
import os
df['audio'] = df['filename'].apply(lambda x: os.path.join("/content/2/cv-other-test/cv-other-test/sample-000000.mp3", x))

In [16]:
df.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration,audio
0,cv-other-test/sample-000000.mp3,you killed all three of them for nothing,0,0,NaN,NaN,NaN,NaN,/content/2/cv-other-test/cv-other-test/sample-...
1,cv-other-test/sample-000001.mp3,will it be warmer in montana,0,0,NaN,NaN,NaN,NaN,/content/2/cv-other-test/cv-other-test/sample-...
2,cv-other-test/sample-000002.mp3,science has been arguing about the zoological ...,0,0,thirties,male,NaN,NaN,/content/2/cv-other-test/cv-other-test/sample-...
3,cv-other-test/sample-000003.mp3,he wasn't asking for help,1,1,NaN,NaN,NaN,NaN,/content/2/cv-other-test/cv-other-test/sample-...
4,cv-other-test/sample-000004.mp3,but he granted the travelers three days,0,0,sixties,male,us,NaN,/content/2/cv-other-test/cv-other-test/sample-...


In [21]:
import torch
import torchaudio
import librosa
import pandas as pd
from transformers import (
    AutoProcessor,
    SeamlessM4TModel,
    Wav2Vec2ForCTC,
    Wav2Vec2Processor
)
import whisper
from jiwer import wer, cer
import os
from tqdm import tqdm  #

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def calculate_wer(reference, hypothesis):
    return wer(reference, hypothesis)

def calculate_cer(reference, hypothesis):
    return cer(reference, hypothesis)

def transcribe_whisper(model, audio_path):
    result = model.transcribe(audio_path, language="english")
    return result["text"].lower()

def transcribe_seamless_m4t(model, processor, audio_path, target_lang="eng", sampling_rate=16000):
    audio, orig_freq = torchaudio.load(audio_path)
    if orig_freq != sampling_rate:
        audio = torchaudio.functional.resample(audio, orig_freq=orig_freq, new_freq=sampling_rate)
    audio = audio.squeeze().numpy()
    audio_inputs = processor(audios=audio, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(**audio_inputs, tgt_lang=target_lang, generate_speech=False)

    if hasattr(output, 'sequences'):
        output_tokens = output.sequences
    else:
        output_tokens = output


    output_tokens = output_tokens.cpu().tolist()


    translated_text = processor.batch_decode(output_tokens, skip_special_tokens=True)[0]
    return translated_text.lower()


def transcribe_wav2vec2(model, processor, audio_path, sampling_rate=16000):
    speech_array, sr = librosa.load(audio_path, sr=sampling_rate)
    inputs = processor(speech_array, sampling_rate=sampling_rate, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

    predicted_ids = torch.argmax(logits, dim=-1)


    predicted_ids = predicted_ids.cpu().tolist()


    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return transcription.lower()

def save_transcription(text, output_path):
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(text)


print("Ładowanie modeli...")


model_whisper = whisper.load_model("base").to(device)


processor_seamless = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-medium")
model_seamless = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-medium").to(device)


MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
processor_wav2vec2 = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model_wav2vec2 = Wav2Vec2ForCTC.from_pretrained(MODEL_ID).to(device)

print("Modele załadowane.")


original_csv_path = "/content/2/cv-other-test.csv"


df = pd.read_csv(original_csv_path).head(200)
print(f"Liczba rekordów w zbiorze danych: {len(df)}")


df['filename'] = df['filename'].str.replace('cv-other-test/', '', regex=False)


df['audio_path'] = df['filename'].apply(lambda x: os.path.join("/content/2/cv-other-test/cv-other-test/", x))


transcription_dir = "/content/2/cv-other-test/transcriptions"
os.makedirs(transcription_dir, exist_ok=True)


test_files = df['audio_path'].head(10)
for file in test_files:
    if os.path.isfile(file):
        print(f"Plik istnieje: {file}")
    else:
        print(f"Plik nie istnieje: {file}")


results = []


print("Przetwarzanie rekordów...")
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    try:
        audio_path = row['audio_path']
        reference = row['text'].lower()

        if not os.path.isfile(audio_path):
            raise FileNotFoundError(f"Plik audio nie został znaleziony: {audio_path}")


        whisper_text = transcribe_whisper(model_whisper, audio_path)


        seamless_text = transcribe_seamless_m4t(model_seamless, processor_seamless, audio_path, target_lang="eng")


        wav2vec2_text = transcribe_wav2vec2(model_wav2vec2, processor_wav2vec2, audio_path)


        whisper_wer = calculate_wer(reference, whisper_text)
        whisper_cer = calculate_cer(reference, whisper_text)

        seamless_wer = calculate_wer(reference, seamless_text)
        seamless_cer = calculate_cer(reference, seamless_text)

        wav2vec2_wer = calculate_wer(reference, wav2vec2_text)
        wav2vec2_cer = calculate_cer(reference, wav2vec2_text)


        base_filename = os.path.splitext(row['filename'])[0]
        transcription_whisper_path = os.path.join(transcription_dir, f"{base_filename}_whisper.txt")
        transcription_seamless_path = os.path.join(transcription_dir, f"{base_filename}_seamless.txt")
        transcription_wav2vec2_path = os.path.join(transcription_dir, f"{base_filename}_wav2vec2.txt")


        save_transcription(whisper_text, transcription_whisper_path)
        save_transcription(seamless_text, transcription_seamless_path)
        save_transcription(wav2vec2_text, transcription_wav2vec2_path)


        results.append({
            "audio_path": audio_path,
            "reference": reference,
            "whisper_text": whisper_text,
            "seamless_m4t_text": seamless_text,
            "wav2vec2_text": wav2vec2_text,
            "whisper_wer": whisper_wer,
            "whisper_cer": whisper_cer,
            "seamless_m4t_wer": seamless_wer,
            "seamless_m4t_cer": seamless_cer,
            "wav2vec2_wer": wav2vec2_wer,
            "wav2vec2_cer": wav2vec2_cer,
            "error": None  # Brak błędu
        })

    except Exception as e:
        print(f"Błąd przetwarzania przykładu {i}: {e}")
        results.append({
            "audio_path": row['audio_path'] if 'audio_path' in row else None,
            "reference": row['text'].lower() if 'text' in row else None,
            "whisper_text": None,
            "seamless_m4t_text": None,
            "wav2vec2_text": None,
            "whisper_wer": None,
            "whisper_cer": None,
            "seamless_m4t_wer": None,
            "seamless_m4t_cer": None,
            "wav2vec2_wer": None,
            "wav2vec2_cer": None,
            "error": str(e)
        })

print("Przetwarzanie zakończone.")


df_results = pd.DataFrame(results)

print("\nWyniki transkrypcji:")
print(df_results.head())

average_wer = df_results[["whisper_wer", "seamless_m4t_wer", "wav2vec2_wer"]].mean()
average_cer = df_results[["whisper_cer", "seamless_m4t_cer", "wav2vec2_cer"]].mean()

print("\nŚrednie WER:")
print(average_wer)

print("\nŚrednie CER:")
print(average_cer)

output_csv = "/content/results_transcription.csv"
df_results.to_csv(output_csv, index=False)
print(f"\nWyniki zostały zapisane do pliku: {output_csv}")


Używane urządzenie: cuda
Ładowanie modeli...


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Modele załadowane.
Liczba rekordów w zbiorze danych: 200
Plik istnieje: /content/2/cv-other-test/cv-other-test/sample-000000.mp3
Plik istnieje: /content/2/cv-other-test/cv-other-test/sample-000001.mp3
Plik istnieje: /content/2/cv-other-test/cv-other-test/sample-000002.mp3
Plik istnieje: /content/2/cv-other-test/cv-other-test/sample-000003.mp3
Plik istnieje: /content/2/cv-other-test/cv-other-test/sample-000004.mp3
Plik istnieje: /content/2/cv-other-test/cv-other-test/sample-000005.mp3
Plik istnieje: /content/2/cv-other-test/cv-other-test/sample-000006.mp3
Plik istnieje: /content/2/cv-other-test/cv-other-test/sample-000007.mp3
Plik istnieje: /content/2/cv-other-test/cv-other-test/sample-000008.mp3
Plik istnieje: /content/2/cv-other-test/cv-other-test/sample-000009.mp3
Przetwarzanie rekordów...


100%|██████████| 200/200 [02:34<00:00,  1.29it/s]

Przetwarzanie zakończone.

Wyniki transkrypcji:
                                          audio_path  \
0  /content/2/cv-other-test/cv-other-test/sample-...   
1  /content/2/cv-other-test/cv-other-test/sample-...   
2  /content/2/cv-other-test/cv-other-test/sample-...   
3  /content/2/cv-other-test/cv-other-test/sample-...   
4  /content/2/cv-other-test/cv-other-test/sample-...   

                                           reference  \
0           you killed all three of them for nothing   
1                       will it be warmer in montana   
2  science has been arguing about the zoological ...   
3                          he wasn't asking for help   
4            but he granted the travelers three days   

                                        whisper_text  \
0          you killed all three of them for nothing.   
1                         will it be warmer montana?   
2   science has been arguing about the zoological...   
3                         he won't ask him for help.  